In [ ]:
import cv2
import mediapipe as mp 
import time 

In [ ]:
class FaceMeshDetector():
    def __init__(self, mode=False, maxFace=2, minDetectionCon=0.5, minTrackCon=0.5):
        self.mode = mode
        self.maxFace = maxFace
        self.minDetectionCon = minDetectionCon
        self.minTrackCon = minTrackCon
        
        self.mpDraw = mp.solutions.drawing_utils
        self.mpFaceMesh = mp.solutions.face_mesh
        self.faceMesh = self.mpFaceMesh.FaceMesh(self.mode, self.maxFace, self.minDetectionCon, self.minTrackCon)
        self.drawSpec = self.mpDraw.DrawingSpec(thickness = 1, circle_radius=2)
    
        
        
    def findFaceMesh (self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.faceMesh.process(imgRGB)
        
        faces = []
        if self.results.multi_face_landmarks: # if something is detected 
            
            for facelms in self.results.multi_face_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, facelms, self.mpFaceMesh.FACE_CONNECTIONS, self.drawSpec, self.drawSpec)
                
                face = []
                for id, lm in enumerate(facelms.landmark): # to loop through the landmarks
                    iw, ih, ic = img.shape
                    x, y = int(lm.x*iw), int(lm.y*ih)
                    
                    face.append([x,y])
                
                faces.append(face)
            
        return img, faces
                
   

In [ ]:
def main():
    cap = cv2.VideoCapture('faceDetectionvideos/1.mp4')
    pTime = 0
    fm = FaceMeshDetector()
    while True:
        success, img = cap.read()
        img, faces = fm.findFaceMesh(img)
        if len(faces) > 0:
            print(len(faces))
        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime

        cv2.putText(img, f'fps:{int(fps)}', (20,70), cv2.FONT_HERSHEY_PLAIN, 3, (255,254,254), 2)
        cv2.imshow('Image', img)
        cv2.waitKey(1)
    

In [ ]:
if __name__ == '__main__':
    main()